In [1]:
import pandas as pd

from typing import Optional, List, Union
import os

from utils.config import get_path_of_directory_with_id

# Helpers

In [2]:
def get_segmentation_metrics_from_experiment(
    experiment_id: str, 
    model_version: str, 
    csv_name: str,
    metrics: List[str]
) -> pd.Series:
    experiment_dir = get_path_of_directory_with_id(experiment_id)
    eval_dir = os.path.join(experiment_dir, "evaluation", model_version)
    metrics_path = os.path.join(eval_dir, csv_name)

    df = pd.read_csv(metrics_path).set_index("patient").loc[["mean", "std"]]
    df_shard = df[metrics].rename(columns={n: f"{n}_mean" for n in df.columns})

    df_shard = (
        df_shard
        .assign(**{
            n.replace("mean", "std"): df_shard.loc["std", n] 
            for n in df_shard.columns
        })
        .loc["mean"]
        .rename(experiment_id)
        .sort_index()
    )
    
    return df_shard

def get_metrics_summary_for_experiments(
    experiment_ids: List[str], 
    model_version: str, 
    csv_names: Union[str, List[str]],
    metrics: List[str]
) -> pd.DataFrame:
    
    if isinstance(csv_names, str):
        csv_names = [csv_names] * len(experiment_ids)

    df = pd.concat([
        get_segmentation_metrics_from_experiment(eid, model_version, csv_name, metrics)
        for eid, csv_name in zip(experiment_ids, csv_names)
    ], axis=1).transpose()
    return df

# Get the summary

In [3]:
model_version : str = "best"
metrics : List[str] = ["sens", "ppv", "dsc", "hd"]
experiment_ids : List[str] = list(range(0, 10)) 
csv_names: List[str] = ["patient_metrics.csv"] * len(experiment_ids)

df = get_metrics_summary_for_experiments(experiment_ids, model_version, csv_names, metrics)
df.index.name = "experiment_id"
df = df.reset_index()

distance_metrics = ["hd", "avc"]
highlight_max_cols = [
    c for c in df.columns 
    if c.endswith("mean") 
    and not any(c.startswith(m) for m in distance_metrics)
]
highlight_min_cols = [
    c for c in df.columns 
    if c.endswith("mean") 
    and any(c.startswith(m) for m in distance_metrics)
]

(
    df
    .style
    .hide_index()
    .highlight_max(
        axis='rows', 
        subset=pd.IndexSlice[:, highlight_max_cols]
    )
    .highlight_min(
        axis='rows', 
        subset=pd.IndexSlice[:, highlight_min_cols]
    )
)

<ipython-input-3-415dfe03fdbf>:23: FutureWarning: this method is deprecated in favour of `Styler.hide(axis='index')`
  df


experiment_id,dsc_mean,dsc_std,hd_mean,hd_std,ppv_mean,ppv_std,sens_mean,sens_std
0,0.229768,0.171500,86.549959,38.202591,0.266496,0.252722,0.267231,0.202413
1,0.179996,0.254553,60.248636,38.778455,0.275920,0.390210,0.133562,0.188885
2,0.472926,0.188022,27.688864,16.693116,0.793934,0.183758,0.428790,0.264368
3,0.404399,0.253552,62.315236,16.242271,0.371704,0.244763,0.456414,0.246309
4,0.343923,0.250862,71.211829,24.600991,0.318969,0.287019,0.648713,0.047284
5,0.496794,0.355772,43.918089,32.540254,0.544893,0.389512,0.480113,0.367996
6,0.577586,0.121632,65.368442,4.119080,0.587627,0.126507,0.568986,0.118902
7,0.579063,0.116465,39.325878,24.203484,0.772886,0.095851,0.499362,0.159419
8,0.345537,0.345537,62.453314,21.956401,0.330205,0.330205,0.362360,0.362360
9,0.217459,0.217181,66.344145,12.706472,0.297336,0.297197,0.204047,0.138473


In [4]:
df.insert(1, "fold", [0, 0, 1, 1, 2, 2, 3, 3, 4, 4])
df.insert(2, "modality", ["ADC", "DWI"] * 5)
df

,experiment_id,fold,modality,dsc_mean,dsc_std,hd_mean,hd_std,ppv_mean,ppv_std,sens_mean,sens_std
0,0,0,ADC,0.229768,0.171500,86.549959,38.202591,0.266496,0.252722,0.267231,0.202413
1,1,0,DWI,0.179996,0.254553,60.248636,38.778455,0.275920,0.390210,0.133562,0.188885
2,2,1,ADC,0.472926,0.188022,27.688864,16.693116,0.793934,0.183758,0.428790,0.264368
3,3,1,DWI,0.404399,0.253552,62.315236,16.242271,0.371704,0.244763,0.456414,0.246309
4,4,2,ADC,0.343923,0.250862,71.211829,24.600991,0.318969,0.287019,0.648713,0.047284
5,5,2,DWI,0.496794,0.355772,43.918089,32.540254,0.544893,0.389512,0.480113,0.367996
6,6,3,ADC,0.577586,0.121632,65.368442,4.119080,0.587627,0.126507,0.568986,0.118902
7,7,3,DWI,0.579063,0.116465,39.325878,24.203484,0.772886,0.095851,0.499362,0.159419
8,8,4,ADC,0.345537,0.345537,62.453314,21.956401,0.330205,0.330205,0.362360,0.362360
9,9,4,DWI,0.217459,0.217181,66.344145,12.706472,0.297336,0.297197,0.204047,0.138473


# Summary for the FOSCAL experiments with the joint embedding model

In [5]:
experiment_ids = [0, 1, 10, 10, 2, 3, 11, 11, 4, 5, 12, 12, 6, 7, 13, 13, 8, 9, 14, 14]
fold = [0, 0, 0, 0, 1, 1, 1, 1, 2, 2, 2, 2, 3, 3, 3, 3, 4, 4, 4, 4]

csv_names, modalities = [], []
first_occurence = True
for experiment_id in experiment_ids:
    if experiment_id < 10:
        csv_names.append("patient_metrics.csv")
        if experiment_id in [0, 2, 4, 6, 8]:
            modalities.append("ADC")
        elif experiment_id in [1, 3, 5, 7, 9]:
            modalities.append("DWI")
    else:
        if first_occurence:
            csv_names.append("adc_patient_metrics.csv")
            modalities.append("ADC (ADC,DWI)")
            first_occurence = False
        else:
            csv_names.append("dwi_patient_metrics.csv")
            modalities.append("DWI (ADC,DWI)")
            first_occurence = True

df = get_metrics_summary_for_experiments(experiment_ids, model_version, csv_names, metrics)
df.index.name = "experiment_id"
df = df.reset_index()

distance_metrics = ["hd", "avc"]
highlight_max_cols = [
    c for c in df.columns 
    if c.endswith("mean") 
    and not any(c.startswith(m) for m in distance_metrics)
]
highlight_min_cols = [
    c for c in df.columns 
    if c.endswith("mean") 
    and any(c.startswith(m) for m in distance_metrics)
]

(
    df
    .style
    .hide_index()
    .highlight_max(
        axis='rows', 
        subset=pd.IndexSlice[:, highlight_max_cols]
    )
    .highlight_min(
        axis='rows', 
        subset=pd.IndexSlice[:, highlight_min_cols]
    )
)

<ipython-input-5-aea085216c85>:40: FutureWarning: this method is deprecated in favour of `Styler.hide(axis='index')`
  df


experiment_id,dsc_mean,dsc_std,hd_mean,hd_std,ppv_mean,ppv_std,sens_mean,sens_std
0,0.229768,0.171500,86.549959,38.202591,0.266496,0.252722,0.267231,0.202413
1,0.179996,0.254553,60.248636,38.778455,0.275920,0.390210,0.133562,0.188885
10,0.420378,0.300300,76.920251,49.574270,0.452474,0.320015,0.398293,0.292790
10,0.037266,0.052703,65.768790,19.325276,0.020823,0.029448,0.177211,0.250614
2,0.472926,0.188022,27.688864,16.693116,0.793934,0.183758,0.428790,0.264368
3,0.404399,0.253552,62.315236,16.242271,0.371704,0.244763,0.456414,0.246309
11,0.641374,0.083858,45.020485,24.708081,0.732624,0.086621,0.584293,0.126877
11,0.414546,0.312607,49.370501,26.892755,0.441319,0.343629,0.392809,0.288914
4,0.343923,0.250862,71.211829,24.600991,0.318969,0.287019,0.648713,0.047284
5,0.496794,0.355772,43.918089,32.540254,0.544893,0.389512,0.480113,0.367996


In [6]:
df.insert(1, "fold", fold)
df.insert(2, "modality", modalities)
df

,experiment_id,fold,modality,dsc_mean,dsc_std,hd_mean,hd_std,ppv_mean,ppv_std,sens_mean,sens_std
0,0,0,ADC,0.229768,0.171500,86.549959,38.202591,0.266496,0.252722,0.267231,0.202413
1,1,0,DWI,0.179996,0.254553,60.248636,38.778455,0.275920,0.390210,0.133562,0.188885
2,10,0,"ADC (ADC,DWI)",0.420378,0.300300,76.920251,49.574270,0.452474,0.320015,0.398293,0.292790
3,10,0,"DWI (ADC,DWI)",0.037266,0.052703,65.768790,19.325276,0.020823,0.029448,0.177211,0.250614
4,2,1,ADC,0.472926,0.188022,27.688864,16.693116,0.793934,0.183758,0.428790,0.264368
5,3,1,DWI,0.404399,0.253552,62.315236,16.242271,0.371704,0.244763,0.456414,0.246309
6,11,1,"ADC (ADC,DWI)",0.641374,0.083858,45.020485,24.708081,0.732624,0.086621,0.584293,0.126877
7,11,1,"DWI (ADC,DWI)",0.414546,0.312607,49.370501,26.892755,0.441319,0.343629,0.392809,0.288914
8,4,2,ADC,0.343923,0.250862,71.211829,24.600991,0.318969,0.287019,0.648713,0.047284
9,5,2,DWI,0.496794,0.355772,43.918089,32.540254,0.544893,0.389512,0.480113,0.367996


In [7]:
df.groupby('modality').mean()

,experiment_id,fold,dsc_mean,dsc_std,hd_mean,hd_std,ppv_mean,ppv_std,sens_mean,sens_std
modality,,,,,,,,,,
ADC,4.0,2.0,0.393948,0.215511,62.654482,21.114436,0.459446,0.236042,0.455216,0.199065
"ADC (ADC,DWI)",12.0,2.0,0.519072,0.202728,42.929300,28.406437,0.530694,0.211246,0.535361,0.193662
DWI,5.0,2.0,0.375542,0.239505,54.430397,24.894187,0.452548,0.283507,0.354699,0.220217
"DWI (ADC,DWI)",12.0,2.0,0.323776,0.234181,54.655287,26.203896,0.425279,0.317708,0.307398,0.230918
